In [9]:
from google import google
import gnp
import codecs
import json
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import boto
import sys, os
import pandas as pd
from TwitterSearch import *
import sunlight
from sunlight import congress
from bs4 import BeautifulSoup
import urllib2
import itertools
import datetime
import configparser
import time
from pattern.en import parsetree
from pattern.en import mood

from googleplaces import GooglePlaces, types, lang
import csv
import pandas as pd
import time
import sys
import requests
import urllib, json

In [26]:
action_categories = ["donation","give","donate"
                     "protest","rally","demonstration","demonstrate",
                     "meetup","huddle","congregate","join",
                     "boycott",
                     "call","email","reach out","senator","representative"
                     "vote",
                     "sign","petition",
                     "town hall","open office","town meeting","townhall",
                     "school board meeting"]

issues = [
"women's rights","planned parenthood",
"black lives matter","african american rights","black rights","civil rights"
"immigration","immigrants","undocumented","border wall","refugees","refugee rights",
"gun control",
"public education",
"LGBTQ rights","LGBTQ","LGBT",
"voting rights","redistricting","voter id laws","voting access","voter access"
         ]

legislators = pd.read_csv('data/static_data/legislators.csv')

mask = (legislators.in_office == 1)

legislators = legislators.loc[mask]

legislators['title_name'] = legislators['full_title'] + ' ' + legislators['firstname'] + ' ' + legislators['lastname']

print legislators.columns
print
print list(legislators['title_name'])[:10]

leg_names = list(legislators['title_name'])
leg_states = list(legislators['state'])

keyword_searches = []

for combination in itertools.product(action_categories, issues):
  keyword_searches.append(list(combination))
  
print keyword_searches[:20]

In [3]:
i = 0
num_page = 3
for search in keyword_searches:
  search_results = google.search(' '.join(search), num_page)
  with open('data4.txt', 'a') as outfile:
    for result in search_results:
      new_record = {
        'source': 'Google',
        'text': result.description,
        'url': result.link,
        'keywords':search,
#         'created_date': ,
        'search_date': datetime.datetime.today().strftime("%m/%d/%Y")
      }
      json.dump(new_record, outfile)
      outfile.write('\n')
      i += 1
      if i % 100 ==0:
        print( result.description  )
        print result.link
        print search



Error accessing: http://www.google.com/search?q=donation+civil+rights&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+civil+rights&start=10&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+civil+rights&start=20&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+women%27s+rights&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+women%27s+rights&start=10&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+women%27s+rights&start=20&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+planned+parenthood&start=0&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+planned+parenthood&start=10&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+planned+parenthood&start=20&num=10&nl=en
Error accessing: http://www.google.com/search?q=donation+black+lives+matter&start=0&num=10&nl=en


KeyboardInterrupt: 

In [19]:
config = configparser.ConfigParser()
config.read('ross.ini')

consumer_key = config['twitter.api']['consumer_key']
consumer_secret = config['twitter.api']['consumer_secret']
access_token = config['twitter.api']['access_token']
access_token_secret = config['twitter.api']['access_token_secret']

AWS_ACCESS_KEY_ID = config['aws.creds']['AWS_ACCESS_KEY_ID']
AWS_SECRET_ACCESS_KEY = config['aws.creds']['AWS_SECRET_ACCESS_KEY']

YOUR_API_KEY = config['google.api']['YOUR_API_KEY']

In [16]:
reload(sys)  
sys.setdefaultencoding('UTF8')


zip_geo = pd.read_csv('~/w210_imwithdata/data/static_data/Zip_lat_long.csv')

zips = []

for index, row in zip_geo.iterrows():
    new_record = {
        'zip': row['ZIP'],
        'lat': row['LAT'],
        'lng': row['LNG']
    }
    zips.append(new_record)

In [17]:
zips[:10]

[{'lat': 18.180554999999998, 'lng': -66.749960999999999, 'zip': 601.0},
 {'lat': 18.361945000000002, 'lng': -67.175596999999996, 'zip': 602.0},
 {'lat': 18.455182999999998, 'lng': -67.119887000000006, 'zip': 603.0},
 {'lat': 18.158345000000001, 'lng': -66.932911000000004, 'zip': 606.0},
 {'lat': 18.295366000000001, 'lng': -67.125135, 'zip': 610.0},
 {'lat': 18.402252999999998, 'lng': -66.711396999999991, 'zip': 612.0},
 {'lat': 18.420411999999999, 'lng': -66.671979000000007, 'zip': 616.0},
 {'lat': 18.445146999999999, 'lng': -66.559696000000002, 'zip': 617.0},
 {'lat': 17.991245000000003, 'lng': -67.153993, 'zip': 622.0},
 {'lat': 18.083361, 'lng': -67.153897000000001, 'zip': 623.0}]

In [ ]:
def GoogPlac(lat,lng,keyword,radius,language,pagetoken,key):
  #making the url
    if pagetoken is str:
        AUTH_KEY = key
        LOCATION = str(lat)+ "," + str(lng)
        KEYWORD =keyword
        RADIUS = radius
        LANGUAGE = language
        PAGETOKEN = pagetoken
        MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
                '?location=%s'
                    '&radius=%s'
                    '&keyword=%s'
                '&language=%s'
                '&page_token=%s'
                '&sensor=false&key=%s') % (LOCATION, RADIUS, KEYWORD, LANGUAGE,PAGETOKEN, AUTH_KEY)
        #grabbing the JSON result
    else:
        AUTH_KEY = key
        LOCATION = str(lat)+ "," + str(lng)
        KEYWORD =keyword
        RADIUS = radius
        LANGUAGE = language
        MyUrl = ('https://maps.googleapis.com/maps/api/place/nearbysearch/json'
                '?location=%s'
                    '&radius=%s'
                    '&keyword=%s'
                '&language=%s'
                '&sensor=false&key=%s') % (LOCATION, RADIUS, KEYWORD, LANGUAGE, AUTH_KEY)
        #grabbing the JSON result
        response = urllib.urlopen(MyUrl)
        jsonRaw = response.read()
        jsonData = json.loads(jsonRaw)
        return jsonData


def GoogDetails(place_id,key):
  #making the url
  AUTH_KEY = key
  PLACEID = place_id
  MyUrl = ('https://maps.googleapis.com/maps/api/place/details/json'
           '?placeid=%s'
           '&sensor=false&key=%s') % (PLACEID, AUTH_KEY)
  #grabbing the JSON result
  response = urllib.urlopen(MyUrl)
  jsonRaw = response.read()
  jsonData = json.loads(jsonRaw)
  return jsonData

google_place_ids = []

i = 0
with open('data_google_places1.txt', 'a') as outfile:
  for search in keyword_searches:
      for i in xrange(0,len(zips),30):
  #         if zips[i]['zip'] in range(60600,60702):
              #if i >= 434:
              # Retry Query for a Max of 3 pages, depending on whether the query has a next_page_token
            for k in range(3):
                # First page will not include a next_page_token in the query
                if k == 0:
                    # Grab the JSON result
                    content = GoogPlac(zips[i]['lat'],zips[i]['lng'],' '.join(search),'50000','en',False,YOUR_API_KEY)
                    # Check for a next page token
                    next_page_token = content.get('next_page_token',None)
                    # Loop through the places and get the details necessary to populate the final records for Placement
                    for place in content['results']:
                        # Query Google Details with the Place ID
                        details = GoogDetails(place['place_id'],YOUR_API_KEY)
                        new_record = {
                                            'source': 'Google',
                                            'keywords': search,
                                            'name': details['result'].get('name','None'),
                                            'google_place_id': place['place_id'],
                                            'address': details['result'].get('formatted_address','None'),
                                            'phone': details['result'].get('formatted_phone_number','None'),
                                            'latitude': place['geometry']['location']['lat'],
                                            'longitude': place['geometry']['location']['lng'],
                                            'google_url': details['result'].get('url','None'),
                                            'website':details['result'].get('website','None'),
                                            }
                        if place['place_id'] not in google_place_ids:
                        # Append the record to the output list
                          json.dump(new_record, outfile)
                          outfile.write('\n')
                          google_place_ids.append(place['place_id'])
                        # Keep track of the output
                        if len(google_place_ids) % 100 == 0:
                            print len(google_place_ids)
                            print new_record
                # If the original query has a next page token, then query the next page
                if k > 0 and next_page_token: 
                    # Add the next page token to the query
                    content = GoogPlac(zips[i]['lat'],zips[i]['lng'],' '.join(search),'50000','en',next_page_token,YOUR_API_KEY)
                    # If there's content, then get the new records
                    if content:
                        next_page_token = content.get('next_page_token',None)
                        # Checking that this part of the code was actually working, so  print page # if afte first page
                        # Google only lets you get to 60 results per query, or 3 pages, so it will never be > 3
                        print 'in page %s of results' % (k+1)
                        for place in content['results']:
                            # Query Google Details with the Place ID
                            details = GoogDetails(place['place_id'],YOUR_API_KEY)
                            new_record = {
                                                'source': 'Google',
                                                'keywords': search,
                                                ''
                                                'name': details['result'].get('name','None'),
                                                'google_place_id': place['place_id'],
                                                'address': details['result'].get('formatted_address','None'),
                                                'phone': details['result'].get('formatted_phone_number','None'),
                                                'latitude': place['geometry']['location']['lat'],
                                                'longitude': place['geometry']['location']['lng'],
                                                'google_url': details['result'].get('url','None'),
                                                'website':details['result'].get('website','None'),
                                                }
                            # Append the record to the output list
                            if place['place_id'] not in google_place_ids:
                            # Append the record to the output list
                              json.dump(new_record, outfile)
                              outfile.write('\n')
                              google_place_ids.append(place['place_id'])
                            # Keep track of the output
                            if len(google_place_ids) % 100 == 0:
                                print len(google_place_ids)
                                print new_record
                    else:
                        break


In [33]:
bucket_name = 'mids-capstone-rzst'
conn = boto.connect_s3(AWS_ACCESS_KEY_ID,AWS_SECRET_ACCESS_KEY)


# bucket = conn.create_bucket(bucket_name,
#     location=boto.s3.connection.Location.DEFAULT)

testfile = "data_google_places1.txt"
print 'Uploading %s to Amazon S3 bucket %s' % \
   (testfile, bucket_name)

def percent_cb(complete, total):
    sys.stdout.write('.')
    sys.stdout.flush()


# k = Key(bucket_name)
# k.key = 'data.txt'
# k.set_contents_from_filename(testfile,
#     cb=percent_cb, num_cb=10)


# s3_connection = boto.connect_s3()
bucket = conn.get_bucket('mids-capstone-rzst')
key = boto.s3.key.Key(bucket, 'data_google_places1.txt')
key.set_contents_from_filename('data_google_places1.txt')
key.set_acl('public-read')